In [227]:
from sqlalchemy import create_engine
import pandas as pd
import re

from datetime import datetime, timedelta
import matplotlib

import numpy as np
import math

import ipycanvas as ic
from dataclasses import dataclass
import colorsys

# Stop Icons (Ignore this notebook)

This notebook is more for fun with colors than for anything else - I'd be interested in finding a nice color representation of routes from stop icons, but have yet to figure out anything that is not painful to look at!

In [3]:
USER = "admin"
PASSWORD = "12345678"
HOST = "dublinbus.cwaixvtk8gyq.us-east-1.rds.amazonaws.com"
PORT = 3306
SCHEMA = "dublin_bus"
CONNECTION_STRING = f"mysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{SCHEMA}"

engine = create_engine(CONNECTION_STRING)
with engine.connect():
    print("Successfully tested DB connection")

Successfully tested DB connection


First, retrieve data linking stops to route lines and agencies.

In [205]:
query = """
    SELECT 
        main,
        api_agency.external_id as agency, 
        api_routenames.`name` as line, 
        direction, 
        stop_id, 
        api_stops.`name` as stop_name, 
        lon, 
        lat 
    FROM 
        api_routestops
        JOIN api_agency ON agency_id = api_agency.id
        JOIN api_routenames ON name_id = api_routenames.id
        JOIN api_stops ON stop_id = api_stops.id
"""

stop_data = pd.read_sql(query, engine)
stop_data.head(2)

,main,agency,line,direction,stop_id,stop_name,lon,lat
0,1,03C,125,0,4624,Johnstown,-6.625139,53.235648
1,1,03C,125,0,2387,Newlands Cross,-6.391883,53.312516


Now, for each stop, note the position & name, and lines that go by.

In [221]:
@dataclass
class Stop:
    name: str
    lon: str
    lat: str
    lines: list
    # These will be set in a later cell
    x: float = 0
    y: float = 0
        
@dataclass 
class StopLine:
    name: str
    direction: int
    agency: str
        
def get_bus_data(agencies="all", lines="all"):

    data = stop_data.values
    stops = dict()

    for main, agency, line, direction, stop_id, stop_name, lon, lat in data:

        stop = stops.get(stop_id, None)

        if stop is None:
            stop = Stop(
                name = stop_name,  
                lon = lon,
                lat = lat,
                lines = list()
            )
            stops[stop_id] = stop

        if main == 0:
            continue
            
        if agencies != "all" and agency not in agencies:
            continue
            
        if lines != "all" and line not in lines:
            continue
            
        stop.lines.append(StopLine(
            name = line,
            direction = direction,
            agency = agency
        ))
                
    # Remove stops which with the filter have no lines.
    for stop_id, stop in list(stops.items()):
        if len(stop.lines) == 0:
            del stops[stop_id]
            
    return stops

(Test)

In [222]:
next(iter(get_bus_data(lines=["46A"]).values()))

Stop(name='Donnybrook Church', lon=-6.23056195593035, lat=53.318330187289, lines=[StopLine(name='46A', direction=0, agency='978')], x=0, y=0)

Once last thing on the data front is that I'd like to draw these stops, and filling in normalized coords would be a good place to start there

In [224]:
def get_data(*args, **kwargs):

    data = get_bus_data(*args, **kwargs)
    
    # No lat/lon will reach 1000
    min_x = min_y = 1000
    max_x = max_y = -1000

    for stop in data.values():

        if stop.lat < min_y: 
            min_y = stop.lat

        if stop.lat > max_y:
            max_y = stop.lat

        if stop.lon < min_x:
            min_x = stop.lon

        if stop.lon > max_x:
            max_x = stop.lon

    for stop in data.values():

        stop.x = (stop.lon - min_x) / (max_x - min_x)
        stop.y = (stop.lat - min_y) / (max_y - min_y)
        
    return data

Drawing the stops, with the option to zoom in on a window.

In [469]:
def draw_all(data, draw_single, x=0.5, y=0.5, zoom=1, w=300, h=None, size=1, bg="#eeeeee"):

    stretch_y = 1.2
    if h is None:
        h = w*stretch_y
    
    canvas = ic.Canvas(width=w, height=h)
    display(canvas)
    if bg is not None:
        canvas.fill_style = bg
        canvas.fill_rect(0, 0, w, h)
    
    screen_w = w
    screen_h = min(h, w)
    window_x = x - 0.5/zoom
    window_y = y - 0.5/zoom
    
    def transform(x, y):
        return (
            screen_w*(zoom*(x - window_x)),
            screen_h*(zoom*(y - window_y))*stretch_y
        )
    
    with ic.hold_canvas(canvas):
        for stop in list(data.values()):
            stop_x, stop_y = transform(stop.x, stop.y)
            draw_single(stop, canvas, stop_x, stop_y, size*zoom)

I'm not sure the coordinates are right - we seem to have lost Cork?
<br>I'm more interested in stop colors though, a rough position is fine for now.m

In [470]:
def simple_red(stop, canvas, x, y, size):
    canvas.fill_style = "#ff0000"
    canvas.fill_circle(x, y, size)

draw_all(get_data(), simple_red, 0.5, 0.5, 1, 300, 400, size=1)

Canvas(height=400, width=300)

Incidentally - 978 is Dublin Bus, which is our primary concern. But they have been outsourcing common routes to GoAhead, so adding them in seems important as well.

In [471]:
def simple_red(stop, canvas, x, y, size):
    canvas.fill_style = "#ff0000"
    canvas.fill_circle(x, y, size)

draw_all(get_data(agencies=["978", "03", "03C"]), simple_red, 0.5, 0.5, 1, 300, 400, size=1)

Canvas(height=400, width=300)

Testing custom colors - just using a hash of the stop name for now.

It's quite chaotic, but that is to be expected.

In [472]:
def random_colors(stop, canvas, x, y, size):
    hue = hash(stop.name) % 180 + 180
    canvas.fill_style = f"hsl({hue}, 85%, 50%)"
    canvas.fill_circle(x, y, size)

draw_all(get_data(agencies=["978"]), random_colors, zoom=2)

Canvas(height=360, width=300)

Random colors per route combination this time.

Definitely more structure here, but it's still mostly a blur of color.

In [473]:
def route_combo(stop, canvas, x, y, size):
    value = "".join([line.name for line in stop.lines])
    hue = hash(value) % 180 + 180
    canvas.fill_style = f"hsl({hue}, 85%, 60%)"
    canvas.fill_circle(x, y, size)

draw_all(get_data(agencies=["978"]), route_combo, zoom=1, w=600, size=1.5)

Canvas(height=720, width=600)

Somethings limiting the color scheme can help. The ones below aren't great, but finding any set of four colors that fit together with equal importance is hard!

In [474]:
def color_bar(colors):
    canvas = ic.Canvas(width=400, height=25)
    display(canvas)
    size = 400/len(colors)
    for i,color in enumerate(colors):
        canvas.fill_style = color
        canvas.fill_rect(i*size, 0, size, 25)

In [475]:
colors = ["#498A42", "#57B8FF", "#FBB13C", "#FE6847"]
color_bar(colors)

Canvas(height=25, width=400)

Attempt 2: Attack of the Pies

In [480]:
def route_pie(stop, canvas, x, y, size):

    # Creating a random starting value for each route combo
    value = "".join([line.name for line in stop.lines])
    i0 = hash(value)
    
    N = len(stop.lines)
    start_angle = math.pi/2
    for i, line in enumerate(stop.lines):
        canvas.fill_style = colors[(i0+i)%len(colors)]
        end_angle = start_angle + 2*math.pi/N
        canvas.fill_arc(x, y, size, start_angle, end_angle)
        start_angle = end_angle
    
draw_all(get_data(agencies=["978"]), route_pie, zoom=1.4, x=0.6, y=0.6, w=600, size=2.2, bg=None)

Canvas(height=720, width=600)

Pie charts seemed like a cool idea in my head. It solves the blur problem somewhat, but it's not something one would want to look at much...

Returning to the unique route combo idea, it might be neat the reduce the color options down to 4ish (4-color theorem)
<br>(Doing this properly would require identifying adjencent segments, though, which would be super awkward.)

In [479]:
def route_combo(stop, canvas, x, y, size):
    value = hash("".join([line.name for line in stop.lines]))
    canvas.fill_style = colors[value % len(colors)]
    canvas.fill_circle(x, y, size)

draw_all(get_data(agencies=["978"]), route_combo, zoom=1, w=600, size=1.5, bg=None)

Canvas(height=720, width=600)

That already looks better to me - and that is just random based on the hash again.
<br>One annoying thing is there seem to be some bits where the routes quickly jump between two colors, which is noisy and annoying.
<br>I wonder is that direction at play here.

In [481]:
def route_combo_outgoing(stop, canvas, x, y, size):
    
    lines = list()
    for line in stop.lines:
        if line.direction == 0:
            lines.append(line.name)
            
    if len(lines) == 0:
        return 
    
    value = hash("".join(lines))
    canvas.fill_style = colors[value % len(colors)]
    canvas.fill_circle(x, y, size)

draw_all(get_data(agencies=["978"]), route_combo_outgoing, zoom=1, w=600, size=1.5, bg=None)

Canvas(height=720, width=600)

Single direction definitely cleans things up - I guess some routes are 1 direction only, or have a different name going th eother direction? Odd.

In [482]:
draw_all(get_data(agencies=["978", "03"]), route_combo_outgoing, zoom=1, w=600, size=1.5, bg=None)

Canvas(height=720, width=600)

One other thing I'd like to try is to mix colors based on routes

First, though, mixing colors is a headache.

I tried to do it algorithmically below, and it gave mixed results (if you'll pardon the pun)

In [461]:
def additive_mix(*colors):
    
    N = len(colors)
    
    r = g = b = 0
    for color in colors:
        r += int(color[1:3], base=16)
        g += int(color[3:5], base=16)
        b += int(color[5:7], base=16)
    
    r = max(0, min(int(r), 255))
    g = max(0, min(int(g), 255))
    b = max(0, min(int(b), 255))
    
    result = "#"
    for value in r,g,b:
        result += hex(value)[2:].rjust(2, "0")
    
    return result

def test_mixer(mixer, colors=("#ff0000", "#00ff00", "#0000ff"), black="#000000"):
    
    class Rect:

        def __init__(self, x, y, w, h, c):
            self.x, self.y = x,y
            self.w, self.h = w,h
            self.c = c

        def draw(self, canvas):
            canvas.fill_style = self.c
            canvas.fill_rect(self.x, self.y, self.w, self.h)

        def __and__(self, other):

            x = max(self.x, other.x)
            y = max(self.y, other.y)
            w = min(self.x + self.w, other.x + other.w) - x
            h = min(self.y + self.h, other.y + other.h) - y
            c = mixer(self.c, other.c)

            return Rect(x, y, w, h, c)
        
    A = Rect(20, 40, 100, 100, colors[0])
    B = Rect(40, 60, 100, 100, colors[1])
    C = Rect(60, 20, 100, 100, colors[2])
    
    canvas = ic.Canvas(width=200, height=200)
    display(canvas)
    
    canvas.fill_style = black
    canvas.fill_rect(0, 0, 200, 200)
   
    A.draw(canvas)
    B.draw(canvas)
    C.draw(canvas)
    
    (A & B).draw(canvas)
    (B & C).draw(canvas)
    (A & C).draw(canvas)
    (A & B & C).draw(canvas)
    

test_mixer(additive_mix)
test_mixer(additive_mix, ("#498A42", "#57B8FF", "#FE6847"), black="#444444")

Canvas(height=200, width=200)

Canvas(height=200, width=200)

A different approach, a custom additive mix with a minimal set of colors.

In [465]:
lookup = {
    "#444444": (0,0,0),  # black
    "#FE6847": (1,0,0),  # red
    "#498A42": (0,1,0),  # green
    "#57B8FF": (0,0,1),  # blue
    "#FFF289": (1,1,0),  # yellow
    "#A0FFFF": (0,1,1),  # cyan
    "#DE85FF": (1,0,1),  # magenta
    "#CCCCCC": (1,1,1),  # white
}

reverse_lookup = dict()
for key, value in lookup.items():
    reverse_lookup[value] = key

flag_colors = [
    reverse_lookup[r,g,b] for r,g,b in [
        (1,0,0), (0,1,0), (0,0,1)
    ]
]
flag_black = reverse_lookup[(0,0,0)]
    
def flag_mix(*colors):
    
    colors = [lookup[color] for color in colors]
    
    r = g = b = 0
    for ri,gi,bi in colors:
        r |= ri
        g |= gi
        b |= bi
        
    return reverse_lookup[r,g,b]
    
test_mixer(flag_mix, flag_colors, flag_black)

Canvas(height=200, width=200)

In [485]:
def line_flag_mix(stop, canvas, x, y, size):
    
    stop_color = flag_black
    for line in stop.lines:
        line_color = flag_colors[hash(line.name) % len(flag_colors)]
        stop_color = flag_mix(stop_color, line_color)
    
    if stop_color == flag_black:
        return

    canvas.fill_style = stop_color
    canvas.fill_circle(x, y, size)

draw_all(get_data(agencies=["978", "03"]), line_flag_mix, zoom=1, w=600, size=1.5, bg=flag_black)

Canvas(height=720, width=600)

Cool imo, but still utterly irrelevant to the app.